# UI

> This nb desings a UI for the blackjack game

In [ ]:
from indo_alem_com_jupyter_notebook.utils import Hand
from indo_alem_com_jupyter_notebook.core import FrenchDeck

import ipywidgets as widgets
from fastcore.basics import patch

You may need to execute `jupyter nbextension enable --py widgetsnbextension` and restart the nb to output the widgets.

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
from IPython.display import display

## Cards UI

The card and deck will be represented as buttons

In [ ]:
def card_button_factory(name: str):
    return widgets.Button(
        disable=True,
        description=name,
        layout=widgets.Layout(height="auto", width="auto"),
    )

In [ ]:
# | exec_doc

In [ ]:
card_button = card_button_factory("test")
card_button

Since the deck is a single button we can define them in a Layout

In [ ]:
class GameLayout(widgets.TwoByTwoLayout):
    def __init__(self, *args, **kwargs):
        self.deck_button = widgets.Button(
            description="Deck (52)",
            tooltip="click me to draw a card!",
            layout=widgets.Layout(height="100%", width="auto"),
        )

        self.reset_button = widgets.Button(
            description="Reset", layout=widgets.Layout(height="auto", width="auto")
        )

        self.displayer = widgets.Text(
            value="Pull a card", disabled=True, layout=widgets.Layout(width="50%")
        )

        super().__init__(
            top_left=self.deck_button,
            top_right=widgets.HBox([self.reset_button, self.displayer]),
            bottom_right=widgets.HBox([]),
            justify_items="center",
            width="50%",
            align_items="center",
        )

In [ ]:
GameLayout()

In [ ]:
@patch
def on_deck_click(self: GameLayout, callback):
    self.top_left.on_click(callback)

In [ ]:
@patch
def on_reset_click(self: GameLayout, callback):
    reset_button = self.top_right.children[0]
    reset_button.on_click(callback)

In [ ]:
@patch
def update_num_cards(self: GameLayout, number: int):
    self.top_left.description = f"Deck ({number})"

In [ ]:
@patch
def clear_hand(self: GameLayout):
    self.bottom_right = widgets.HBox([])

In [ ]:
@patch
def update_hand(self: GameLayout, hand: Hand):
    self.bottom_right = widgets.HBox(
        [card_button_factory(str(card.rank)) for card in hand]
    )
    self.update_num_cards(52 - len(hand))

    value = hand.value()  # type: ignore

    if value > 21:
        self.displayer.value = "Game over"
    elif value == 21:
        self.displayer.value = "You won!"
    else:
        self.displayer.value = str(value)

In [ ]:
class BlackJack:
    def __init__(self):
        self.view = GameLayout()
        self.view.on_deck_click(self.on_deck_clicked)
        self.view.on_reset_click(self.on_reset_clicked)
        self._start()

    def _start(self):
        self.deck = FrenchDeck()
        self.deck.shuffle()
        self.hand = Hand()

    def on_deck_clicked(self, event):
        card = self.deck.draw()
        self.hand += card
        self.view.update_hand(self.hand)

    def on_reset_clicked(self, event):
        self._start()
        self.view.update_hand(self.hand)

    def __repr__(self):
        display(self.view)
        return ""

In [ ]:
blackjack = BlackJack()
blackjack

Run voila over here! with `voila 12_ui_for_jupyter_notebooks.ipynb`